請根據各自組別資料, 利用程式完成下列的要求:
1. 處理missing data (刪除與填補資料)
2. 每個特徵與Label的敘述統計
3. 特徵縮放
4. 類別特徵轉換
5. 訓練與測試split data

# Load Data

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
# 選擇剛剛下載好的 kaggle.json 檔案

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d camnugent/california-housing-prices

  0% 0.00/400k [00:00<?, ?B/s]
100% 400k/400k [00:00<00:00, 65.0MB/s]


In [7]:
! mkdir california-housing-prices

# 將剛剛載下來的.zip壓縮檔解壓縮進資料夾裡
! unzip california-housing-prices.zip -d california-housing-prices

Archive:  california-housing-prices.zip
  inflating: california-housing-prices/housing.csv  


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import missingno as msno

from sklearn.preprocessing import OneHotEncoder ,LabelEncoder ,MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split

sns.set(style="whitegrid")
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 2)
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

# EDA

In [8]:
df = pd.read_csv('/content/drive/MyDrive/大三/下學期/機器學習應用/Datasets/california_housing_train/california_housing_train_original.csv')
# df = pd.read_csv('/content/california_housing_train_original.csv')
df # 20640 rows × 10 columns

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.33,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.30,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.26,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.64,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.85,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.56,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.56,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.70,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.87,84700.0,INLAND


In [ ]:
df.info()

# Processing data

In [ ]:
df.rename(columns={'housing_median_age' :'housing_age'}, inplace=True)
df.rename(columns={'median_income' :'income'}, inplace=True)
df.rename(columns={'median_house_value' :'house_value'}, inplace=True)

df

# Missing Value

In [ ]:
msno.matrix(df)

In [ ]:
df.isnull().sum()

total_bedrooms有207筆的缺失數值，其餘欄位都沒有缺失值

In [ ]:
df.describe()

依作業要求，這邊就只處理缺失值，異常值之後再做處理

In [ ]:
mean_total_bedrooms = df['total_bedrooms'].mean()

# 平均值填入total_bedrooms的空值
df['total_bedrooms'].fillna(mean_total_bedrooms, inplace=True)

df.describe()

In [ ]:
df.isnull().sum()

# Statistics

In [ ]:
df.describe(include=['object'])

In [ ]:
df.describe()

# Feature Engineering

In [19]:
numeric_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# 數值做StandardScaler
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# 類別做LabelEncoder
label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [ ]:
df

In [ ]:
df.describe()

經過標準化的轉換後，可看到各欄位的均值趨近於0，方差為1



In [ ]:
# use_columns = ['ocean_proximity']
# df_copy = df.copy()

# label_encoder = LabelEncoder()

# for col in use_columns:
#     df_copy[col + '_encoded'] = label_encoder.fit_transform(df[col])

#     original_values = df_copy[col].unique()
#     encoded_values = df_copy[col + '_encoded'].unique()
#     print(f"Original values for column '{col}': {original_values}")
#     print(f"Encoded values for column '{col}': {encoded_values}")
#     print()

# for col in use_columns:
#     df[col] = label_encoder.fit_transform(df[col])

Original values for column 'ocean_proximity': ['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']

Encoded values for column 'ocean_proximity': [3 0 1 4 2]

ocean_proximity：房子相對海洋的位置（類別：「<1H 海洋:0」、「內陸:1」、「靠近海洋:4」、「靠近海灣:3」、「島嶼:2」）

# Split data

In [23]:
X = df.drop('house_value', axis=1)
y = df['house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (16512, 9)
X_test shape: (4128, 9)
y_train shape: (16512,)
y_test shape: (4128,)
